In [1]:
!nvidia-smi

Wed Jun 29 07:09:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
! pip install -q datasets
! pip install torch==1.6.0
! pip install transformers
! pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
import numpy as np
import os
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/kaggle//us-patent/train.csv')
eval_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/kaggle//us-patent/test.csv')

In [5]:
df['input'] = 'TEXT1:' + df.context + '; TEXT2:' + df.target + '; ANC1:' + df.anchor
df.head()

,id,anchor,target,context,score,input
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,TEXT1:A47; TEXT2:abatement of pollution; ANC1:...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,TEXT1:A47; TEXT2:act of abating; ANC1:abatement
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,TEXT1:A47; TEXT2:active catalyst; ANC1:abatement
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,TEXT1:A47; TEXT2:eliminating process; ANC1:aba...
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,TEXT1:A47; TEXT2:forest region; ANC1:abatement


In [6]:
from datasets import Dataset,DatasetDict
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [7]:
from transformers import AlbertTokenizer
import torch

albert_model = 'tals/albert-base-mnli'
tokenizer = AlbertTokenizer.from_pretrained(albert_model)

In [8]:
tokenizer.tokenize("A platypus is an ornithorhynchus anatinus.")

['▁a',
 '▁platy',
 'pus',
 '▁is',
 '▁an',
 '▁',
 'ornith',
 'o',
 'rhynch',
 'us',
 '▁an',
 'at',
 'inus',
 '.']

In [9]:
tokenizer("A platypus is an ornithorhynchus anatinus.")

{'input_ids': [2, 21, 26138, 8032, 25, 40, 13, 22210, 111, 29507, 267, 40, 721, 9585, 9, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
def tok_func(x): return tokenizer(x['input'])

In [11]:
tok_ds = ds.map(tok_func, batched = True)

Parameter 'function'=<function tok_func at 0x7f79d3b9e0e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/37 [00:00<?, ?ba/s]

In [12]:
tok_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36473
})

In [13]:
tok_ds = tok_ds.rename_columns({'score':'labels'})

In [14]:
from numpy.random import normal,seed,uniform
np.random.seed(2022)
train_and_val = tok_ds.train_test_split(0.25, seed = 2022)
train_and_val

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [15]:
eval_df['input'] = 'TEXT1:' + df.context + '; TEXT2:' + df.target + '; ANC1:' + df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched = True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
eval_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36
})

In [17]:
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim

class MyDataSet(Data.Dataset):
  def __init__(self, data, labels):
    self.data = data
    self.labels = labels
    self.tokenizer = AlbertTokenizer.from_pretrained(albert_model)

  def __getitem__(self, idx):
    text = self.data[idx]
    label = self.labels[idx]
    inputs = self.tokenizer(text, return_tensors = 'pt', padding = 'max_length', max_length = 30, truncation = True)
    input_ids = inputs.input_ids.squeeze(0)
    token_type_ids = inputs.token_type_ids.squeeze(0)
    attention_mask = inputs.attention_mask.squeeze(0)
    return input_ids, token_type_ids, attention_mask, label

  def __len__(self):
    return len(self.data)

In [18]:
data = tok_ds['input']
label = tok_ds['labels']

dataset = MyDataSet(data, label)
dataloader = Data.DataLoader(dataset, batch_size = 128, shuffle = True)

In [19]:
data[:5]

['TEXT1:A47; TEXT2:abatement of pollution; ANC1:abatement',
 'TEXT1:A47; TEXT2:act of abating; ANC1:abatement',
 'TEXT1:A47; TEXT2:active catalyst; ANC1:abatement',
 'TEXT1:A47; TEXT2:eliminating process; ANC1:abatement',
 'TEXT1:A47; TEXT2:forest region; ANC1:abatement']

In [20]:
label[:5]

[0.5, 0.75, 0.25, 0.5, 0.0]

In [21]:
import torch.nn as nn
from transformers import AlbertModel
class MyModel(nn.Module):
  def __init__(self):
    super(MyModel, self).__init__()
    self.albert = AlbertModel.from_pretrained(albert_model)
    self.linear = nn.Linear(768, 1)

  def forward(self, input_ids, token_type_ids, attention_mask):
    output = self.albert(input_ids, token_type_ids, attention_mask).pooler_output
    output = self.linear(output)

    return output

In [22]:
device = torch.device('cuda')
model = MyModel().to(device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-6)

Some weights of the model checkpoint at tals/albert-base-mnli were not used when initializing AlbertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
for ep in range(8):
  
  for input_ids, token_type_ids, attention_mask, label in dataloader:
    input_ids, token_type_ids, attention_mask, label = input_ids.to(device), token_type_ids.to(device), attention_mask.to(device), label.to(device)
    label = label.float().unsqueeze(1)
    # print(label.shape)
    pred = model(input_ids, token_type_ids, attention_mask)
    # pred = pred.squeeze(1)
    # print(pred.shape)
    loss = loss_fn(pred, label)
    print('epoch:',ep,loss.item())

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

epoch: 0 0.12883836030960083
epoch: 0 0.12067151069641113
epoch: 0 0.08489428460597992
epoch: 0 0.10047774761915207
epoch: 0 0.1031571626663208
epoch: 0 0.07995064556598663
epoch: 0 0.07560727000236511
epoch: 0 0.07053934037685394
epoch: 0 0.0795411467552185
epoch: 0 0.08579257130622864
epoch: 0 0.0678732693195343
epoch: 0 0.07264082133769989
epoch: 0 0.07898133248090744
epoch: 0 0.08026933670043945
epoch: 0 0.06493256986141205
epoch: 0 0.06563663482666016
epoch: 0 0.050594352185726166
epoch: 0 0.07354505360126495
epoch: 0 0.06309893727302551
epoch: 0 0.07468453049659729
epoch: 0 0.09492796659469604
epoch: 0 0.08193773776292801
epoch: 0 0.06039257347583771
epoch: 0 0.06428742408752441
epoch: 0 0.06699003279209137
epoch: 0 0.0780075266957283
epoch: 0 0.06878389418125153
epoch: 0 0.06717263162136078
epoch: 0 0.07652581483125687
epoch: 0 0.05366459861397743
epoch: 0 0.07375900447368622
epoch: 0 0.07323499023914337
epoch: 0 0.061703428626060486
epoch: 0 0.05855439603328705
epoch: 0 0.06367

In [24]:
class MyDataTestSet(Data.Dataset):
  def __init__(self, data):
    self.data = data
    self.tokenizer = AlbertTokenizer.from_pretrained(albert_model)

  def __getitem__(self, idx):
    text = self.data[idx]
    inputs = self.tokenizer(text, return_tensors = 'pt', padding = 'max_length', max_length = 30, truncation = True)
    input_ids = inputs.input_ids.squeeze(0)
    token_type_ids = inputs.token_type_ids.squeeze(0)
    attention_mask = inputs.attention_mask.squeeze(0)
    return input_ids, token_type_ids, attention_mask

  def __len__(self):
    return len(self.data)

In [25]:
test_data = eval_ds['input']

dataset = MyDataTestSet(test_data)
test_dataloader = Data.DataLoader(dataset, batch_size = 1, shuffle = True)

In [26]:
for _ in range(len(test_dataloader)):
  test_pred = []
  for input_ids, token_type_ids, attention_mask in test_dataloader:
    
    input_ids, token_type_ids, attention_mask = input_ids.to(device), token_type_ids.to(device), attention_mask.to(device)
    predict = model(input_ids, token_type_ids, attention_mask)
    test_pred.append(predict.item())

In [27]:
test_pred

[0.4048767685890198,
 0.31984034180641174,
 0.43249183893203735,
 0.44883251190185547,
 0.22682541608810425,
 0.21807044744491577,
 0.32805177569389343,
 0.31257379055023193,
 0.29930251836776733,
 0.3747761845588684,
 0.3077618479728699,
 0.3358863592147827,
 0.43610385060310364,
 0.3567951023578644,
 0.3324335813522339,
 0.5046437382698059,
 0.32346034049987793,
 0.3788154125213623,
 0.3980962336063385,
 0.25342997908592224,
 0.6444774866104126,
 0.29964369535446167,
 0.25241872668266296,
 0.24220693111419678,
 0.3097335994243622,
 0.4735998213291168,
 0.5227141380310059,
 0.39471209049224854,
 0.4736317992210388,
 0.2753794193267822,
 0.24418304860591888,
 0.41699397563934326,
 0.2612352967262268,
 0.6279081106185913,
 0.25278666615486145,
 0.49580931663513184]

In [28]:
import datasets
submission = datasets.Dataset.from_dict(
    {
       'id': eval_ds['id'],
       'score': test_pred
    }
)
submission

Dataset({
    features: ['id', 'score'],
    num_rows: 36
})

In [29]:
submission.to_csv('albert.csv', index = False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1324